# INF8215 - Intelligence artif.: méthodes et algorithmes 
## Automne 2019 - TP2 - Recherche Adversarielle 
### Membres de l'équipe
    - Membre 1
    - Membre 2
    - Membre 3




## Directives de remise
Le travail sera réalisé avec la  même équipe que pour le TP1. Vous remettrez ce fichier nommé TP2\_NomDuMembre1\_NomDuMembre2\_NomDuMembre3.ipynb dans la boîte de remise sur moodle. 

Tout devra être remis avant le **6 novembre à 23h55**. Tout travail en retard sera pénalisé
d’une valeur de 10\% par jour de retard.

## Barème
Partie 1: 25 points

Partie 2: 35 points

Partie 3: 15 points

Partie 4: 15 points

Partie 5: 10 points

Pour un total de 100 points possibles sur 100 points.


## Recherche Adversarielle

La recherche adversarielle est une méthode d’intelligence artificielle qui est utilisée pour prédire les actions d’un adversaire dans un jeu vidéo. Il consiste à construire un arbre avec les actions possibles suivi des réactions de l’adversaire à ces actions. Pour choisir l’action optimale, il existe plusieurs méthodes. Une très commune est la recherche minimax. Elle évalue la valeur des actions et maximise la valeur de celle du joueur et minimise la valeur de celle de l’adversaire.

### Rush Hour

Vous avez vu le jeu de puzzle, Rush Hour, dans le TP précédent. Dans le cadre de ce TP, un adversaire a été ajouté pour transformer Rush Hour en jeu adversarielle. À chaque tour, l’adversaire ajoute une roche dans la grille. Cette roche disparaît après 1 tour et empêche les voitures de se déplacer. Son but est d’obliger le joueur à faire le plus de coups possible et de l’empêcher de bouger complètement. Le joueur peut déplacer une voiture, selon les règles vues dans le tp précédent, par tour. Cette version de Rush Hour est un jeu zero-sum déterministe.

![Rushhour](https://i.stack.imgur.com/NzztF.jpg)

### But

Votre but dans ce TP est de développer un algorithme minimax pour permettre à votre AI de terminer le niveau de Rush Hour **avec un minimun de mouvement**. Une version complétée des classes Rush Hour et State est fournie pour vous aider à faire le TP. Ces classes vont être réutilisées pour représenter les états à évaluer. Vous pouvez les remplacer par vos propres implémentations en Python (vous pouvez aussi réécrire le TP dans le langage de votre choix).  

## 1. Implémentation de Rush Hour (25pts)
Les classes suivantes sont complètes pour représenter les différents états d’une partie de Rush Hour normale.
Il manque la représentation des coups de l’adversaire : vous auriez à implémenter cette partie-là dans les classes touchées.

L’état du stationnement au début et à tout moment du jeu est représenté par un objet de la classe **State** qui contient les champs suivants : 

- `pos` : un vecteur indiquant la position de chaque véhicule sur la grille (la valeur représente la première case occupée par la voiture);

- `c` : l’index de la voiture déplacée pour atteindre l’état courant à partir de l’état précédent;

- `d` : la direction du déplacement fait à partir de l’état précédent pour atteindre l’état courant (+1 : vers la droite ou vers le bas, -1 : vers la gauche ou vers le haut);

- `prev` : l’état précédent

- `score` : le score associé à l'état

- `rock` : la position de la roche dans la grille (ligne,colonne)

In [2]:
import numpy as np
import math
import copy
from collections import deque

class State:
    
    """
    Contructeur d'un état initial
    """
    def __init__(self, pos):
        """
        pos donne la position de la voiture i dans sa ligne ou colonne (première case occupée par la voiture);
        """
        self.pos = np.array(pos)
        
        """
        c,d et prev premettent de retracer l'état précédent et le dernier mouvement effectué
        """
        self.c = self.d = self.prev = None
        
        self.nb_moves = 0
        self.score = 0
        
        # TODO
        self.rock = None

    """
    Constructeur d'un état à partir du mouvement (c,d)
    """
    def move(self, c, d, rh):
        s = State(self.pos)
        s.prev = self
        s.pos[c] += d
        s.c = c
        s.d = d
        s.rock = self.rock
        s.nb_moves = self.nb_moves + 1
        s.score = self.score + s.score_state(rh) - 1
        return s

    def put_rock(self, rock_pos):
        newState = State(self.pos)
        newState.nb_moves = self.nb_moves
        newState.prev = self
        newState.c = self.c
        newState.d = self.d
        newState.score = self.score
        newState.rock = rock_pos
        return newState
            
    def score_state(self,rh):
        nbMouvementsMin1 = 0
        nbMouvementsMin2 = 0
        nbMouvementsMin3 = 0
        end = 0
        
        for i in range(1, rh.nbcars):
            if not rh.horiz[i]: #Si la voiture entrave la ligne de la voiture rouge
                if(rh.move_on[i] >= self.pos[0] + rh.length[0]): 
                    if(self.pos[i] <= 2):
                        if(self.pos[i] + rh.length[i] > 2): 
                            if(rh.length[i] == 2):
                                nbMouvementsMin1 += 1 #Une voiture de longueur 2 induit 1 seul mouvement obligatoire pour libérer la voie
                            else:
                                nbMouvementsMin1 += 3 - self.pos[i] #Une voiture de longueur 3 induit 1, 2 ou 3 mouvements obligatoires
                            
                            for j in range(1,rh.nbcars):
                                if rh.horiz[j]:
                                    #if(rh.move_on[j] == self.pos[i]-1 or rh.move_on[j] == self.pos[i] + rh.length[i]):
                                    if self.pos[j] <= rh.move_on[i] and self.pos[j] + rh.length[j] > rh.move_on[i]:
                                        nbMouvementsMin2 += 1
                                        
            else:
                if self.pos[i] >= self.pos[0] + rh.length[0] : 
                    nbMouvementsMin3 += 1 
                                            
        if self.pos[0] - 4 == 0:
            end = 1000
        
        return self.pos[0] - 4 + end - nbMouvementsMin1 - nbMouvementsMin2 -nbMouvementsMin3
        
    def success(self):
        return self.pos[0] == 4
    
    def __eq__(self, other):
        if not isinstance(other, State):
            return NotImplemented
        if len(self.pos) != len(other.pos):
            print("les états n'ont pas le même nombre de voitures")
        
        return np.array_equal(self.pos, other.pos)
    
    def __hash__(self):
        h = 0
        for i in range(len(self.pos)):
            h = 37*h + self.pos[i]
        return int(h)
    
    def __lt__(self, other):
        return (self.score) < (other.score)


La représentation du problème est faite par la classe **Rushhour** qui contient les champs suivants :

- `nbcars` : le nombre de voitures dans la grille;
- `horiz` : un vecteur contenant un bool indiquant si la voiture bouge horizontalement ou pas;
- `length`: un vecteur contenant la longueur de chaque voiture (2 ou 3);
- `move_on` : un vecteur contenant le numéro de la ligne ou la colonne où se trouve la voiture (0-5);
- `color` : un vecteur contenant le string indiquant la couleur de chaque voiture;
- `free_pos` : une matrice 6x6 contenant un bool permettant de savoir si la case est libre

Toutes les informations pour une voiture se trouvent dans le même index dans chacun des vecteurs.

La fonction *`print_pretty_grid()`* sert à imprimer la grille en utilisant la première lettre des couleurs pour vous aider à mieux visualiser le problème.

In [3]:
class Rushhour:
    
    def __init__(self, horiz, length, move_on, color=None):
        self.nbcars = len(horiz)
        self.horiz = horiz
        self.length = length
        self.move_on = move_on
        self.color = color
        
        self.free_pos = None
    
    def init_positions(self, state):
        self.free_pos = np.ones((6, 6), dtype=bool)
        if state.rock :
            self.free_pos[state.rock[0],state.rock[1]] = False
        for i in range(self.nbcars):
            if self.horiz[i]:
                self.free_pos[self.move_on[i], state.pos[i]:state.pos[i]+self.length[i]] = False
            else:
                self.free_pos[state.pos[i]:state.pos[i]+self.length[i], self.move_on[i]] = False
    
    def possible_moves(self, state):
        self.init_positions(state)
        new_states = []
        for i in range(self.nbcars):
            if self.horiz[i]:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[self.move_on[i], state.pos[i]+self.length[i]]:
                    new_states.append(state.move(i, +1, self))
                if state.pos[i] > 0 and self.free_pos[self.move_on[i], state.pos[i] - 1]:
                    new_states.append(state.move(i, -1, self))
            else:
                if state.pos[i]+self.length[i]-1 < 5 and self.free_pos[state.pos[i]+self.length[i], self.move_on[i]]:
                    new_states.append(state.move(i, +1, self))
                if state.pos[i] > 0 and self.free_pos[state.pos[i] - 1, self.move_on[i]]:
                    new_states.append(state.move(i, -1, self))
        return new_states
    
    def possible_rock_moves(self, state):
        self.init_positions(state)
        new_states =[]
        currentRock = state.rock
        for i in range(6):
            if i != 2 :
                for j in range(6):
                    if currentRock:
                        if i != currentRock[0] and j != currentRock[1] and self.free_pos[i,j]:
                            new_states.append(state.put_rock((i,j)))
                    else:
                        if self.free_pos[i,j]:
                            new_states.append(state.put_rock((i,j)))
        return new_states
    

    def print_pretty_grid(self, state):
        self.init_positions(state)
        grid= np.chararray((6, 6))
        grid[:]='-'
        for car in range(self.nbcars):
            for pos in range(state.pos[car], state.pos[car] +self.length[car]):
                if self.horiz[car]:
                    grid[self.move_on[car]][pos] = self.color[car][0]
                else:
                    grid[pos][self.move_on[car]] = self.color[car][0]
        if state.rock:
            grid[state.rock] = 'x'
        print(grid)

### 1.1 Implémentation

1. Modifier la classe State pour qu'elle contienne la position de la roche ainsi que la fonction *`put_rock()`* pour ajouter une nouvelle roche et enlever l'ancienne. Les roches sont représentées par un tuple (ligne,colonne). La fonction retourne un nouvel objet State.

2. Modifier la fonction *`init_position()`* de la classe Rushhour pour qu'elle tienne compte de l'emplacement des roches.

3. Implémenter la fonction *`possible_rock_moves()`* qui permet de trouver tous les coups possibles de l'adversaire (avec la roche) à partir de l'état courant dans la classe Rushhour. L'adversaire ne peut ni mettre de roche sur la ligne 2 ni mettre deux roches consécutives sur la même ligne ou colonne.

Utiliser la fonction *testRocks()* pour vérifier que vous avez bien pris en compte les roches. Vous devriez avoir ces résultats:

```
État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'x' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False False False False False]
 [False False False False False False]
 [False False False False  True False]]

```

Utiliser la fonction *testPossibleRockMoves()* pour vérifier que vous trouvez bien tous les coups possibles pour l'adversaire. Vous devriez trouver 7 et 3 mouvements possibles.

In [4]:
def testRocks():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s0 = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    
    s1= s0.put_rock((4,4))
    s2 = s1.put_rock((3,2)) 
    
    print("État initial")
    rh.print_pretty_grid(s0)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 4-4")
    rh.print_pretty_grid(s1)
    print(rh.free_pos)
    print('\n')
    
    print("Roche 3-2")
    rh.print_pretty_grid(s2)
    print(rh.free_pos)
    print('\n')

testRocks()

État initial
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'-' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False  True False]
 [False False False False  True False]]


Roche 4-4
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'-']
 [b'-' b'r' b'-' b'v' b'v' b'v']
 [b'v' b'r' b'n' b'n' b'x' b'b']
 [b'v' b'b' b'b' b'b' b'-' b'b']]
[[ True  True False False False False]
 [False  True False False False False]
 [False False False False  True  True]
 [ True False  True False False False]
 [False False False False False False]
 [False False False False  True False]]


Roche 3-2
[[b'-' b'-' b'v' b'j' b'j' b'j']
 [b'o' b'-' b'v' b'v' b'b' b'b']
 [b'o' b'r' b'r' b'v' b'-' b'

In [5]:
def testPossibleRockMoves():
    rh = Rushhour([True, False, True, False, False, True, False, True, False, True, False, True],
                 [2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 2, 3],
                 [2, 2, 0, 0, 3, 1, 1, 3, 0, 4, 5, 5],
                 ["rouge", "vert clair", "jaune", "orange", "violet clair", "bleu ciel", "rose", "violet", "vert", "noir", "beige", "bleu"])
    s = State([1, 0, 3, 1, 1, 4, 3, 4, 4, 2, 4, 1])
    sols = rh.possible_rock_moves(s)
    print(len(sols))
    s1 = s.put_rock((3,4))
    sols = rh.possible_rock_moves(s1)
    print(len(sols))

testPossibleRockMoves()
  

7
3


## 2. Implémentation d'une recherche minimax (35pts)
L'algorithme minimax est l'implémentation la plus commune pour résoudre un jeu adversarielle avec des mouvements limités définis. Cet algorithme est basé sur le principe de construire un arbre avec les différents états possibles selon les actions de chaque joueur (AI et adversaire). L'algorithme veut maximiser les étages où le AI joue et minimiser les étages où l'adversaire joue. Dans ce cas-ci, l'AI déplace les voitures et l'adversaire place les roches. Dans le cadre du TP, l'algorithme n'explore pas jusqu'aux feuilles de l'arbre. Il s'arrête à une profondeur donnée.

La résolution du problème est faite par la classe **MiniMaxSearch** qui contient les champs suivants:

- `rushhour`: la classe Rushhour qui contient les informations sur la grille;
- `state`: la classe State avec l'état actuel de la grille;
- `search_depth`: la profondeur maximale d'exploration de l'algorithme;


In [6]:
import random

class MiniMaxSearch:
    
      
    
    def __init__(self, rushHour, initial_state, search_depth):
        self.rushhour = rushHour
        self.state = initial_state
        self.search_depth = search_depth
        self.minimax2_state_visited = 0          #nombre d'état visité durant le minimax normal
        self.minimax2_visited_history=[]         #stockage, par recherche du meileur move, du nombre d'état visité pour le minimax normal 
        self.minimax_pruning_state_visited = 0   #nombre d'état visité durant le minimax pruned 
        self.minimax_pruning_visited_history=[]  #stockage, par recherche du meileur move, du nombre d'état visité pour le minimax pruned
        self.expectimax_type = "random"          #type d'expectimax, de base aléatoire
    
    def change_expectimax_type (self,x):
        self.expectimax_type = x
        return
    
    def get_minimax2_visited_history(self):
        return self.minimax2_visited_history
    
    def get_minimax_pruning_visited_history(self):
        return self.minimax_pruning_visited_history
    
    def minimax_1(self, current_depth, current_state):
        
        if current_depth == self.search_depth or current_state.success():
            return (current_state.score, current_state.c, current_state.d)
        
        best_move = [float('-Inf')]
        best_posible_moves = []
        
        for child in self.rushhour.possible_moves(current_state):
            move = self.minimax_1(current_depth + 1, child)
            
            if best_move[0] == move[0] :
                best_possible_moves.append([move[0], child.c, child.d])
                
            elif best_move[0] < move[0] : 
                best_move = [move[0], child.c, child.d]
                best_possible_moves = [best_move]
           
        return random.choice(best_possible_moves)
    
    
    def minimax_2(self, current_depth, current_state, is_max):
        
        self.minimax2_state_visited += 1
        
        if current_depth == self.search_depth or current_state.success():
            return (current_state.score, current_state.c, current_state.d,current_state.rock)
        
        if is_max:  # on va maximier le score
            
            best_move = [float('-Inf')]
            best_posible_moves = []

            for child in self.rushhour.possible_moves(current_state):
                move = self.minimax_2(current_depth + 1, child, False)

                if best_move[0] == move[0] :
                    best_possible_moves.append([move[0], child.c, child.d, child.rock])

                elif best_move[0] < move[0] : 
                    best_move = [move[0], child.c, child.d, child.rock]
                    best_possible_moves = [best_move]

            return random.choice(best_possible_moves)
    
    
        else:     # on va minimiser le score
            
            worst_move = [float('Inf')]
            worst_posible_moves = []

            for child in self.rushhour.possible_rock_moves(current_state):
                move = self.minimax_2(current_depth + 1, child, True)

                if worst_move[0] == move[0] :
                    worst_posible_moves.append([move[0], child.c, child.d, child.rock])

                elif worst_move[0] > move[0] : 
                    worst_move = [move[0], child.c, child.d, child.rock]
                    worst_posible_moves = [worst_move]

            return random.choice(worst_posible_moves)


    def minimax_pruning(self, current_depth, current_state, is_max, alpha, beta):
        
        self.minimax_pruning_state_visited += 1
        
        if current_depth == self.search_depth or current_state.success():
            return (current_state.score, current_state.c, current_state.d,current_state.rock)
        
        if is_max:
            
            best_move = [float('-Inf')]
            best_posible_moves = []

            for child in self.rushhour.possible_moves(current_state):
                move = self.minimax_pruning(current_depth + 1, child, False,alpha, beta)

                if best_move[0] == move[0] :
                    best_possible_moves.append([move[0], child.c, child.d, child.rock])

                elif best_move[0] < move[0] : 
                    best_move = [move[0], child.c, child.d, child.rock]
                    best_possible_moves = [best_move]
                
                alpha = max(alpha,move[0])
                if beta <= alpha:
                    break
                
            return random.choice(best_possible_moves)
    
    
        else:
            
            worst_move = [float('Inf')]
            worst_posible_moves = []

            for child in self.rushhour.possible_rock_moves(current_state):
                move = self.minimax_pruning(current_depth + 1, child, True, alpha, beta)

                if worst_move[0] == move[0] :
                    worst_posible_moves.append([move[0], child.c, child.d, child.rock])

                elif worst_move[0] > move[0] : 
                    worst_move = [move[0], child.c, child.d, child.rock]
                    worst_posible_moves = [worst_move]
                
                beta = min(beta, move[0])
                if beta <= alpha:
                    break
                
            return random.choice(worst_posible_moves)
        

    def expectimax(self, current_depth, current_state, is_max):
        
        if current_depth == self.search_depth or current_state.success():
            return (current_state.score, current_state.c, current_state.d,current_state.rock)
        
        if is_max: # on maximise le score
            
            best_move = [float('-Inf')]
            best_posible_moves = []

            for child in self.rushhour.possible_moves(current_state):
                move = self.expectimax(current_depth + 1, child, False)

                if best_move[0] == move[0] :
                    best_possible_moves.append([move[0], child.c, child.d, child.rock])

                elif best_move[0] < move[0] : 
                    best_move = [move[0], child.c, child.d, child.rock]
                    best_possible_moves = [best_move]

            return random.choice(best_possible_moves)
    
    
        else: # on prend une action de manière aléatoire
            
            random_posible_moves = []
            score=[]
            
            for child in self.rushhour.possible_rock_moves(current_state):
                move = self.expectimax(current_depth + 1, child, True)
                random_posible_moves.append(move)
                score.append(move[0]) 
            
            if self.expectimax_type == "random":
                return random.choice(random_posible_moves)
            
            """
            # A finir quand on sera fixé sur l'évolution du score
            
            elif self.expectimax_type == "pessimistic":
                score_sum = sum(score)
                probability = [x/score_sum for x in score] 
                return np.random.choice(random_possible_moves,1, p=probability)
            
            elif self.expectimax_type == "pessimistic":
            """

    def decide_best_move_1(self):
        best_move = self.minimax_1(0, self.state)
        c = best_move[1]
        d = best_move[2]        
        return self.state.move(c,d,self.rushhour)

    
    def decide_best_move_2(self, is_max):
        best_move = self.minimax_2(0, self.state,is_max)
        self.minimax2_visited_history.append(self.minimax2_state_visited)
        self.minimax2_state_visited = 0
        
        if is_max:
            c = best_move[1]
            d = best_move[2]        
            return self.state.move(c,d,self.rushhour)
        
        else : 
            return self.state.put_rock(best_move[3])
        

    def decide_best_move_pruning(self, is_max):
        best_move = self.minimax_pruning(0, self.state,is_max,float('-Inf'),float('Inf'))
        self.minimax_pruning_visited_history.append(self.minimax_pruning_state_visited)
        self.minimax_pruning_state_visited = 0
        
        if is_max:
            c = best_move[1]
            d = best_move[2]        
            return self.state.move(c,d,self.rushhour)
        
        else : 
            return self.state.put_rock(best_move[3])
        
    
    def decide_best_move_expectimax(self, is_max):
        best_move = self.expectimax(0, self.state,is_max)
        if is_max:
            c = best_move[1]
            d = best_move[2]        
            return self.state.move(c,d,self.rushhour)
        
        else : 
            return self.state.put_rock(best_move[3]) 
    
    def solve(self, state, is_singleplayer, minimax = 'regular'):
        """
        minimax est le type d'algo de résolution on va utiliser.
        De base on utilise le "regular" = minimax2.
        Sinon on peut utiliser "pruning" = minimax_pruning ou "expectimax" = expectimax.
        """
        self.state = state
        move_nb = 0
        self.rushhour.print_pretty_grid(self.state)
        
        if is_singleplayer :
            while not self.state.success():
                print()
                move_nb += 1
                self.state = self.decide_best_move_1()
                print("Move",move_nb," : ", end='')
                self.print_move(True,self.state)
                #print("Score : ",self.state.score)
                self.rushhour.print_pretty_grid(self.state)
                
        else:
            is_max = True
            while not self.state.success():
                
                if is_max:
                    print()
                    move_nb += 1
                    
                    if minimax == 'pruning' : 
                        self.state = self.decide_best_move_pruning(is_max)
                        
                    elif minimax == 'regular': 
                        self.state = self.decide_best_move_2(is_max)
                    
                    elif minimax == 'expectimax':
                        self.state = self.decide_best_move_expectimax(is_max)
                    
                    print("Move",move_nb," : ", end='')
                    self.print_move(True,self.state)
                    #print("Score : ",self.state.score)
                    self.rushhour.print_pretty_grid(self.state)
                    
                else:
                    print()
                    
                    if minimax == 'pruning' : 
                        self.state = self.decide_best_move_pruning(is_max)
                        
                    elif minimax == 'regular': 
                        self.state = self.decide_best_move_2(is_max)
                    
                    elif minimax == 'expectimax':
                        self.state = self.decide_best_move_expectimax(is_max)
                    
                    self.print_move(False,self.state)
                    #print("Score : ",self.state.score)
                    self.rushhour.print_pretty_grid(self.state)
                    
                is_max = not(is_max)
        
        return
    
    def print_move(self, is_max, state):
        if is_max:
            if self.rushhour.horiz[state.c]:
                if state.d == 1:
                    direction = "la droite"
                else:
                    direction = "la gauche"
            else:
                if state.d == 1:
                    direction = "le bas"
                else:
                    direction = "le haut"
            #On affiche successiment les mouvements permettant de passer d'un état au suivant
            print("Voiture ", self.rushhour.color[state.c], "vers ", direction)
        else:
            print("Rocher en ",state.rock)
        return  


### 2.1 Implémentation simple
1. Implémenter la fonction *`score_state()`* de la classe **State**. Elle affecte la valeur de l'état à son paramètre score. L'état n'est pas nécessairement final. Utiliser l'heuristique qui vous semble la plus pertinente.

2. Implémenter la fonction  *`minimax_1()`*. Cette fonction contient la logique de l'algorithme minimax pour un seul joueur et retourne le meilleur coup à prendre à partir de l'état courant.

3. Implémenter la fonction *`decide_best_move_1()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à un joueur.

4. Implémenter la fonction *`solve()`*. Cette fonction résout un problème de Rush Hour avec le nombre minimal de coups.

4. Implémenter la fonction *`print_move()`* pour imprimer le coup fait. Ex. "Voiture orange vers le haut" ou "Roche dans la case 2-1".

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

Utiliser la fonction _test_print_move()_ pour vérifier votre implémentation de _print_move()_. 

In [7]:
def test_print_move():
    rh = Rushhour([True], [2], [2], ["rouge"])
    s = State([0])
    s = s.put_rock((3,1)) # Roche dans la case 3-1
    s = s.move(0, 1, rh) # Voiture rouge vers la droite
    algo = MiniMaxSearch(rh, s, 1)
    algo.print_move(True, s)
    algo.print_move(False, s)

test_print_move()

Voiture  rouge vers  la droite
Rocher en  (3, 1)


Pour vous aider dans l'implémentation de votre heuristique, 3 exemples de problèmes de Rush Hour sont fournis.

In [8]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,2)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True)

%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]

Move 1  : Voiture  bleu vers  le haut
[[b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]

Move 2  : Voiture  vert vers  le bas
[[b'-' b'-' b'-' b'-' b'-' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]

Move 3  : Voiture  vert vers  le bas
[[b'-' b'-' b'-' b'-' b'-' b'b']
 [b'-' b'-' b'-' b'-

In [8]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]

Move 1  : Voiture  jaune vers  le bas
Score :  -11
[[b'v' b'v' b'-' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]

Move 2  : Voiture  lime vers  la gauche
Score :  -22
[[b'v' b'v' b'-' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'l' b'l' b'-']
 [b'o' b'-' b'e' b'e' b'e' b'-']]

Move 3  : Voiture  lime vers  la gauche
Score :  -32
[[b'v' b

In [20]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,4)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, True)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 1  : Voiture  3 vers  le bas
Score :  -19
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'-' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'3' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 2  : Voiture  3 vers  le bas
Score :  -35
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'-' b'-' b'5']
 [b'r' b'r' b'-' b'-' b'4' b'5']
 [b'6' b'6' b'6' b'3' b'4' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 3  : Voiture  rouge vers  la droite
Score :  -50
[[b'1' b'-' b'-' 


Move 38  : Voiture  2 vers  la droite
Score :  -250
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 39  : Voiture  2 vers  la gauche
Score :  -255
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 40  : Voiture  2 vers  la droite
Score :  -260
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 41  : Voiture  2 vers  la gauche
Score :  -265
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9'

Move 75  : Voiture  2 vers  la gauche
Score :  -435
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 76  : Voiture  2 vers  la droite
Score :  -440
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 77  : Voiture  2 vers  la gauche
Score :  -445
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 78  : Voiture  2 vers  la droite
Score :  -450
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' 


Move 113  : Voiture  2 vers  la gauche
Score :  -625
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 114  : Voiture  2 vers  la droite
Score :  -630
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 115  : Voiture  2 vers  la gauche
Score :  -635
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 116  : Voiture  2 vers  la droite
Score :  -640
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' 

Move 148  : Voiture  1 vers  le bas
Score :  -800
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 149  : Voiture  2 vers  la gauche
Score :  -805
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 150  : Voiture  2 vers  la droite
Score :  -810
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 151  : Voiture  2 vers  la gauche
Score :  -815
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9'

...


Move 269  : Voiture  1 vers  le haut
Score :  -1405
[[b'1' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'-' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 270  : Voiture  1 vers  le bas
Score :  -1410
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 271  : Voiture  2 vers  la gauche
Score :  -1415
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 272  : Voiture  2 vers  la droite
Score :  -1420
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b

Move 304  : Voiture  2 vers  la droite
Score :  -1580
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 305  : Voiture  1 vers  le haut
Score :  -1585
[[b'1' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'-' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 306  : Voiture  1 vers  le bas
Score :  -1590
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 307  : Voiture  1 vers  le haut
Score :  -1595
[[b'1' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'-' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9'

Move 337  : Voiture  1 vers  le haut
Score :  -1745
[[b'1' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'-' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 338  : Voiture  1 vers  le bas
Score :  -1750
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 339  : Voiture  1 vers  le haut
Score :  -1755
[[b'1' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'-' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 340  : Voiture  1 vers  le bas
Score :  -1760
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'

Move 374  : Voiture  1 vers  le bas
Score :  -1930
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 375  : Voiture  1 vers  le haut
Score :  -1935
[[b'1' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'-' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 376  : Voiture  1 vers  le bas
Score :  -1940
[[b'-' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 377  : Voiture  1 vers  le haut
Score :  -1945
[[b'1' b'2' b'2' b'2' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'-' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'

KeyboardInterrupt: 

### 2.2 Implémentation adversarielle

1. Implémenter la fonction  *`minimax_2()`*. Cette fonction contient la logique de l'algorithme minimax pour deux joueurs et retourne le meilleur coup à prendre à partir de l'état courant.

2. Implémenter la fonction *`decide_best_move_2()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.

3. Modifier la fonction *`solve()`* pour qu'elle puisse résoudre des problèmes à deux joueurs.

4. Modifier la fonction *`score_state()`* de la classe **State** si elle ne tient pas déjà compte des roches.

**N.B**: Vous pouvez modifier les déclarations des fonctions ainsi qu'ajouter des fonctions tant que l'algorithme fonctionne.

 Le nombre donné de coups pour les 3 exemples suivants correspond à une partie sans adversaire. Le nombre de coups de plus dépend de l'implémentation de votre heuristique.

In [21]:
# Solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]

839  states visited with regular minimax
Move 1  : Voiture  bleu vers  le haut
Score :  -8
[[b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]

1400  states visited with regular minimax
Rocher en  (3, 2)
Score :  -8
[[b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'-']
 [b'-' b'o' b'x' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]

37

In [0]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False) 
%time

In [0]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

## 3. Élagage Alpha-Beta (15pts)
Pour accélérer la prise de décision, un élagage est effectué sur l'arbre. Cela permet de ne pas parcourir tout l'arbre. L'élagage alpha-beta est l'implémentation la plus fréquente pour résoudre ce genre de problème. En passant des valeurs d’alpha et de bêta récursivement, l'algorithme connaît la meilleure valeur trouvée à date. Les branches jugées non intéressantes, c'est-à-dire qu'elles ne peuvent pas produire un meilleur résultat que celui déjà trouvé, sont coupées dans le processus.

 
### 3.1 Implémentation

1. Implémenter la fonction *`minimax_pruning()`* pour qu'elle n'explore pas des branches inutiles lors d'une partie à deux joueurs.
2. Implémenter la fonction *`decide_best_move_pruning()`*. Cette fonction trouve et exécute le meilleur coup pour une partie à deux joueurs.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Vérifier que cette nouvelle méthode visite bel et bien moins de noeuds. **Montrer cette information dans un tableau**.

In [9]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, 'pruning')
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]

Move 1  : Voiture  rouge vers  la droite
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'r' b'r' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]

Rocher en  (4, 2)
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'r' b'r' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'x' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]

Move 2  : Voiture  bleu vers  le haut
[[b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [

In [78]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, 'pruning') 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]

Move 1  : Voiture  jaune vers  le bas
[[b'v' b'v' b'-' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]

Rocher en  (5, 5)
[[b'v' b'v' b'-' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'x']]

Move 2  : Voiture  bleu vers  le bas
[[b'v' b'v' b'-' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm'

Move 24  : Voiture  rouge vers  la droite
[[b'm' b'v' b'v' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'x' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (0, 3)
[[b'm' b'v' b'v' b'x' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 25  : Voiture  rouge vers  la gauche
[[b'm' b'v' b'v' b'x' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'r' b'r' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (3, 2)
[[b'm' b'v' b'v' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'r' b'r' b'-' b'j']
 [b'o' b'-' b'x' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 26  : Voiture  rouge vers  la droite
[[b'm' b'v' b'v' b'-' b'-' b'-']
 [b'm

Move 54  : Voiture  vert vers  la droite
[[b'm' b'-' b'v' b'v' b'-' b'j']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'x']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (1, 3)
[[b'm' b'-' b'v' b'v' b'-' b'j']
 [b'm' b'-' b'-' b'x' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 55  : Voiture  jaune vers  le bas
[[b'm' b'-' b'v' b'v' b'-' b'-']
 [b'm' b'-' b'-' b'x' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (0, 1)
[[b'm' b'x' b'v' b'v' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 56  : Voiture  vert vers  la droite
[[b'm' b'x' b'-' b'v' b'v' b'-']
 [b'm' b'-

Move 79  : Voiture  jaune vers  le bas
[[b'm' b'-' b'v' b'v' b'-' b'-']
 [b'm' b'-' b'x' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (0, 1)
[[b'm' b'x' b'v' b'v' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 80  : Voiture  vert vers  la droite
[[b'm' b'x' b'-' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (1, 4)
[[b'm' b'-' b'-' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'x' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 81  : Voiture  vert vers  la gauche
[[b'm' b'-' b'v' b'v' b'-' b'-']
 [b'm' b'-

Rocher en  (0, 3)
[[b'm' b'v' b'v' b'x' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 107  : Voiture  rouge vers  la gauche
[[b'm' b'v' b'v' b'x' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'r' b'r' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (5, 5)
[[b'm' b'v' b'v' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'r' b'r' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'x']]

Move 108  : Voiture  rouge vers  la droite
[[b'm' b'v' b'v' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'x']]

Rocher en  (0, 3)
[[b'm' b'v' b'v' b'x' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' 

...

Move 584  : Voiture  vert vers  la gauche
[[b'm' b'v' b'v' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'x' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (0, 3)
[[b'm' b'v' b'v' b'x' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 585  : Voiture  jaune vers  le haut
[[b'm' b'v' b'v' b'x' b'-' b'j']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (4, 1)
[[b'm' b'v' b'v' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'x' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 586  : Voiture  jaune vers  le bas
[[b'm' b'v' b'v' b'-' b'-' b'-']
 [b'm' b

Rocher en  (3, 1)
[[b'm' b'v' b'v' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'r' b'r' b'-' b'j']
 [b'o' b'x' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 608  : Voiture  rouge vers  la droite
[[b'm' b'v' b'v' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'x' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (0, 3)
[[b'm' b'v' b'v' b'x' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 609  : Voiture  jaune vers  le haut
[[b'm' b'v' b'v' b'x' b'-' b'j']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (4, 1)
[[b'm' b'v' b'v' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'-' b'-' b'

Rocher en  (3, 4)
[[b'm' b'-' b'-' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'x' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 631  : Voiture  vert vers  la gauche
[[b'm' b'-' b'v' b'v' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'x' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (1, 1)
[[b'm' b'-' b'v' b'v' b'-' b'-']
 [b'm' b'x' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 632  : Voiture  vert vers  la gauche
[[b'm' b'v' b'v' b'-' b'-' b'-']
 [b'm' b'x' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (0, 3)
[[b'm' b'v' b'v' b'x' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'

Rocher en  (0, 3)
[[b'm' b'v' b'v' b'x' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 655  : Voiture  jaune vers  le haut
[[b'm' b'v' b'v' b'x' b'-' b'j']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (4, 1)
[[b'm' b'v' b'v' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'x' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 656  : Voiture  jaune vers  le bas
[[b'm' b'v' b'v' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'x' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (0, 3)
[[b'm' b'v' b'v' b'x' b'-' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']

KeyboardInterrupt: 

In [28]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, 'pruning')
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

116  states visited with prunning
Move 1  : Voiture  2 vers  la gauche
Score :  -18
[[b'1' b'-' b'2' b'2' b'2' b'-']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

101  states visited with prunning
Rocher en  (1, 4)
Score :  -18
[[b'1' b'-' b'2' b'2' b'2' b'-']
 [b'1' b'-' b'-' b'3' b'x' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

107  states visi

68  states visited with prunning
Move 24  : Voiture  9 vers  la droite
Score :  -190
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

136  states visited with prunning
Rocher en  (1, 1)
Score :  -190
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'x' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

77  states visited with prunning
Move 25  : Voiture  9 vers  la gauche
Score :  -195
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'x' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

86  states visited with prunning
Rocher en  (0, 3)
Score :  -195
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']


112  states visited with prunning
Rocher en  (0, 3)
Score :  -327
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

68  states visited with prunning
Move 50  : Voiture  9 vers  la droite
Score :  -333
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

136  states visited with prunning
Rocher en  (4, 1)
Score :  -333
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'x' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

89  states visited with prunning
Move 51  : Voiture  9 vers  la gauche
Score :  -338
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']

101  states visited with prunning
Rocher en  (0, 3)
Score :  -470
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

68  states visited with prunning
Move 76  : Voiture  9 vers  la droite
Score :  -476
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

136  states visited with prunning
Rocher en  (1, 2)
Score :  -476
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'x' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

89  states visited with prunning
Move 77  : Voiture  9 vers  la gauche
Score :  -481
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'x' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']

113  states visited with prunning
Move 101  : Voiture  9 vers  la gauche
Score :  -613
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'x' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

124  states visited with prunning
Rocher en  (0, 3)
Score :  -613
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

68  states visited with prunning
Move 102  : Voiture  9 vers  la droite
Score :  -619
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

136  states visited with prunning
Rocher en  (1, 2)
Score :  -619
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'x' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'

...

136  states visited with prunning
Rocher en  (1, 2)
Score :  -3050
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'x' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

89  states visited with prunning
Move 545  : Voiture  9 vers  la gauche
Score :  -3055
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'x' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

101  states visited with prunning
Rocher en  (0, 3)
Score :  -3055
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

68  states visited with prunning
Move 546  : Voiture  9 vers  la droite
Score :  -3061
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r'

77  states visited with prunning
Move 569  : Voiture  9 vers  la gauche
Score :  -3187
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'x' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

86  states visited with prunning
Rocher en  (0, 3)
Score :  -3187
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

68  states visited with prunning
Move 570  : Voiture  9 vers  la droite
Score :  -3193
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

136  states visited with prunning
Rocher en  (4, 1)
Score :  -3193
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' 

89  states visited with prunning
Move 595  : Voiture  9 vers  la gauche
Score :  -3330
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'x' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

99  states visited with prunning
Rocher en  (0, 3)
Score :  -3330
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

68  states visited with prunning
Move 596  : Voiture  9 vers  la droite
Score :  -3336
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

136  states visited with prunning
Rocher en  (1, 2)
Score :  -3336
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'x' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' 

86  states visited with prunning
Rocher en  (0, 3)
Score :  -3462
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

68  states visited with prunning
Move 620  : Voiture  9 vers  la droite
Score :  -3468
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

136  states visited with prunning
Rocher en  (1, 2)
Score :  -3468
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'x' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

89  states visited with prunning
Move 621  : Voiture  9 vers  la gauche
Score :  -3473
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'x' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' 

KeyboardInterrupt: 

In [11]:
#Comparaison du minimax normal et du minimax avec pruning : 
import statistics as stat

rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])

algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)

algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False, 'pruning')
print()
print("States visités par action pour le minimax normal :")
print(algo.get_minimax2_visited_history())
print("Moyenne :",round(stat.mean(algo.get_minimax2_visited_history())))
print()
print("States visités par action pour le minimax avec pruning :")
print(algo.get_minimax_pruning_visited_history())
print("Moyenne :",round(stat.mean(algo.get_minimax_pruning_visited_history())))

%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]

Move 1  : Voiture  bleu vers  le haut
[[b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]

Rocher en  (1, 0)
[[b'-' b'-' b'-' b'-' b'v' b'b']
 [b'x' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]

Move 2  : Voiture  vert vers  le bas
[[b'-' b'-' b'-' b'-' b'-' b'b']
 [b'x' b'-' b'-' b'-' b'v' b'b']
 [b'-'

## 4. Expectimax (15 pts)
L'expectimax est une variante de la recherche minimax. Elle est utilisée lorsqu'il y a une composante de hasard lors de la prise de décision. Depuis le début du TP, il a été supposé que l'adversaire prenait toujours le coup optimal. Maintenant, l'adversaire va faire un coup aléatoirement. Pour résoudre ce cas-là, la recherche expectimax sera utilisée.

### 4.1 Implémentation 

1. Implémenter la fonction *`minimax_expectimax()`* pour qu'elle implémente l'algorithme de recherche. Considérer une probabilité égale pour tous les coups.
2. Implémenter la fonction *`decide_best_move_expectimax()`*. Cette fonction trouve et exécute le meilleur coup pour une partie avec un adversaire imprévisible.
3. Modifier la fonction *`solve()`* pour qu'elle utilise cette nouvelle méthode.
4. Modifier les valeurs de probabilités associées aux coups pour donner une vision optimiste et une vision pessimiste à l'AI. Comparer la performance du AI entre les 3 visions (aléatoire, optimiste, pessimiste). Attention l'adversaire continue à prendre ses décisions au hasard. **Montrer ces informations dans un tableau et les analyser**. 

In [83]:
# solution optimale: 9 moves
rh = Rushhour([True, False, False, False, True],
                 [2, 3, 2, 3, 3],
                 [2, 4, 5, 1, 5],
                 ["rouge", "vert", "bleu", "orange", "jaune"])
s = State([1, 0, 1, 3, 2])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False,'expectimax')
%time

[[ True  True  True  True False  True]
 [ True  True  True  True False False]
 [ True False False  True False False]
 [ True False  True  True  True  True]
 [ True False  True  True  True  True]
 [ True False False False False  True]]
[[b'-' b'-' b'-' b'-' b'v' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]

Move 1  : Voiture  vert vers  le bas
[[b'-' b'-' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'v' b'-']
 [b'-' b'o' b'-' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]

Rocher en  (4, 2)
[[b'-' b'-' b'-' b'-' b'-' b'-']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-' b'r' b'r' b'-' b'v' b'b']
 [b'-' b'o' b'-' b'-' b'v' b'-']
 [b'-' b'o' b'x' b'-' b'-' b'-']
 [b'-' b'o' b'j' b'j' b'j' b'-']]

Move 2  : Voiture  bleu vers  le haut
[[b'-' b'-' b'-' b'-' b'-' b'b']
 [b'-' b'-' b'-' b'-' b'v' b'b']
 [b'-'

In [84]:
# solution optimale: 16 moves
rh = Rushhour([True, True, False, False, True, True, False, False],
                 [2, 2, 3, 2, 3, 2, 3, 3],
                 [2, 0, 0, 0, 5, 4, 5, 3],
                 ["rouge", "vert", "mauve", "orange", "emeraude", "lime", "jaune", "bleu"])
s = State([1, 0, 1, 4, 2, 4, 0, 1])
algo = MiniMaxSearch(rh, s, 3) 
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False) 
%time

[[False False  True  True  True False]
 [False  True  True False  True False]
 [False False False False  True False]
 [False  True  True False  True  True]
 [False  True  True  True False False]
 [False  True False False False  True]]
[[b'v' b'v' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'-']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]

Move 1  : Voiture  jaune vers  le bas
[[b'v' b'v' b'-' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'-' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]

Rocher en  (4, 3)
[[b'v' b'v' b'-' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'm' b'r' b'r' b'b' b'-' b'j']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'x' b'l' b'l']
 [b'o' b'-' b'e' b'e' b'e' b'-']]

Move 2  : Voiture  vert vers  la droite
[[b'-' b'v' b'v' b'-' b'-' b'-']
 [b'm' b'-' b'-' b'b' b'-' b'j']
 [b

Move 34  : Voiture  vert vers  la gauche
[[b'm' b'-' b'-' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'x' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (0, 2)
[[b'm' b'-' b'x' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 35  : Voiture  vert vers  la droite
[[b'm' b'-' b'x' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (5, 4)
[[b'm' b'-' b'-' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'x' b'-']]

Move 36  : Voiture  vert vers  la gauche
[[b'm' b'-' b'-' b'v' b'v' b'-']
 [b'm' b

Rocher en  (1, 3)
[[b'm' b'-' b'-' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'x' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 64  : Voiture  vert vers  la gauche
[[b'm' b'-' b'-' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'x' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (0, 2)
[[b'm' b'-' b'x' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 65  : Voiture  vert vers  la droite
[[b'm' b'-' b'x' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (3, 4)
[[b'm' b'-' b'-' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'-' b'j'

Rocher en  (0, 2)
[[b'm' b'-' b'x' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 93  : Voiture  vert vers  la droite
[[b'm' b'-' b'x' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (1, 3)
[[b'm' b'-' b'-' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'x' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 94  : Voiture  vert vers  la gauche
[[b'm' b'-' b'-' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'x' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (0, 2)
[[b'm' b'-' b'x' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j'

Rocher en  (0, 2)
[[b'm' b'-' b'x' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 123  : Voiture  vert vers  la droite
[[b'm' b'-' b'x' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (3, 1)
[[b'm' b'-' b'-' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'x' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 124  : Voiture  vert vers  la gauche
[[b'm' b'-' b'-' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'x' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (0, 2)
[[b'm' b'-' b'x' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'

...

Rocher en  (0, 2)
[[b'm' b'-' b'x' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 1371  : Voiture  vert vers  la droite
[[b'm' b'-' b'x' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (4, 1)
[[b'm' b'-' b'-' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'x' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 1372  : Voiture  vert vers  la gauche
[[b'm' b'-' b'-' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'x' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (0, 2)
[[b'm' b'-' b'x' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' 

Rocher en  (1, 4)
[[b'm' b'-' b'-' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'x' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 1402  : Voiture  vert vers  la gauche
[[b'm' b'-' b'-' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'x' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (0, 2)
[[b'm' b'-' b'x' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 1403  : Voiture  vert vers  la droite
[[b'm' b'-' b'x' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (5, 4)
[[b'm' b'-' b'-' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'-' 

Rocher en  (0, 2)
[[b'm' b'-' b'x' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 1433  : Voiture  vert vers  la droite
[[b'm' b'-' b'x' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (1, 4)
[[b'm' b'-' b'-' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'x' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 1434  : Voiture  vert vers  la gauche
[[b'm' b'-' b'-' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'x' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (0, 2)
[[b'm' b'-' b'x' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' 

Rocher en  (0, 2)
[[b'm' b'-' b'x' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 1463  : Voiture  vert vers  la droite
[[b'm' b'-' b'x' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'-' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (3, 1)
[[b'm' b'-' b'-' b'-' b'v' b'v']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'x' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Move 1464  : Voiture  vert vers  la gauche
[[b'm' b'-' b'-' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' b'j']
 [b'm' b'-' b'-' b'r' b'r' b'j']
 [b'o' b'x' b'-' b'b' b'-' b'j']
 [b'o' b'-' b'-' b'b' b'l' b'l']
 [b'e' b'e' b'e' b'b' b'-' b'-']]

Rocher en  (0, 2)
[[b'm' b'-' b'x' b'v' b'v' b'-']
 [b'm' b'-' b'-' b'-' b'-' 

KeyboardInterrupt: 

In [85]:
# solution optimale: 14 moves
rh = Rushhour([True, False, True, False, False, False, True, True, False, True, True],
                 [2, 2, 3, 2, 2, 3, 3, 2, 2, 2, 2],
                 [2, 0, 0, 3, 4, 5, 3, 5, 2, 5, 4],
                 ["rouge", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10"])
s = State([0, 0, 3, 1, 2, 1, 0, 0, 4, 3, 4])
algo = MiniMaxSearch(rh, s,3)
algo.rushhour.init_positions(s)
print(algo.rushhour.free_pos)
algo.solve(s, False)
%time

[[False  True  True False False False]
 [False  True  True False  True False]
 [False False  True False False False]
 [False False False  True False False]
 [ True  True False  True False False]
 [False False False False False  True]]
[[b'1' b'-' b'-' b'2' b'2' b'2']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 1  : Voiture  2 vers  la gauche
[[b'1' b'-' b'2' b'2' b'2' b'-']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'-' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Rocher en  (4, 0)
[[b'1' b'-' b'2' b'2' b'2' b'-']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' b'r' b'-' b'3' b'4' b'5']
 [b'6' b'6' b'6' b'-' b'4' b'5']
 [b'x' b'-' b'8' b'-' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 2  : Voiture  2 vers  la gauche
[[b'1' b'2' b'2' b'2' b'-' b'-']
 [b'1' b'-' b'-' b'3' b'-' b'5']
 [b'r' 

Rocher en  (4, 1)
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'x' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Move 35  : Voiture  9 vers  la gauche
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'x' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Rocher en  (0, 3)
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 36  : Voiture  9 vers  la droite
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Rocher en  (4, 1)
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'

Rocher en  (0, 3)
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 58  : Voiture  9 vers  la droite
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Rocher en  (1, 1)
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'x' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Move 59  : Voiture  9 vers  la gauche
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'x' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Rocher en  (0, 3)
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'

Move 81  : Voiture  9 vers  la gauche
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'x' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Rocher en  (0, 3)
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 82  : Voiture  9 vers  la droite
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Rocher en  (4, 1)
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'x' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Move 83  : Voiture  9 vers  la gauche
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' 

Move 105  : Voiture  9 vers  la gauche
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'x' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Rocher en  (0, 3)
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 106  : Voiture  9 vers  la droite
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Rocher en  (4, 1)
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'x' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Move 107  : Voiture  9 vers  la gauche
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'

...

Move 239  : Voiture  9 vers  la gauche
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'x' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Rocher en  (0, 3)
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 240  : Voiture  9 vers  la droite
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Rocher en  (4, 1)
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'x' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Move 241  : Voiture  9 vers  la gauche
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'

Move 261  : Voiture  9 vers  la gauche
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'x' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Rocher en  (0, 3)
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 262  : Voiture  9 vers  la droite
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Rocher en  (1, 2)
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'x' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Move 263  : Voiture  9 vers  la gauche
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'

Move 285  : Voiture  9 vers  la gauche
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'x' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Rocher en  (0, 3)
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 286  : Voiture  9 vers  la droite
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Rocher en  (3, 4)
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'x' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Move 287  : Voiture  9 vers  la gauche
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'

Move 308  : Voiture  9 vers  la droite
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Rocher en  (1, 2)
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'x' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'-' b'9' b'9']]

Move 309  : Voiture  9 vers  la gauche
[[b'2' b'2' b'2' b'-' b'4' b'-']
 [b'1' b'-' b'x' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Rocher en  (0, 3)
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'-' b'-' b'4' b'5']
 [b'1' b'-' b'-' b'r' b'r' b'5']
 [b'6' b'6' b'6' b'3' b'-' b'5']
 [b'-' b'-' b'8' b'3' b'1' b'1']
 [b'7' b'7' b'8' b'9' b'9' b'-']]

Move 310  : Voiture  9 vers  la droite
[[b'2' b'2' b'2' b'x' b'4' b'-']
 [b'1' b'-' b'

KeyboardInterrupt: 

## 5. Comparaison (10pts)
Comparez la performance la recherche minimax avec la recherche expectimax lorsque l'adversaire est optimale et lorsque l'adversaire est aléatoire. Pour un adversaire aléatoire, prenez une vision neutre. Roulez plusieurs fois les algorithmes (vous pouvez aussi reprendre les tests du TP1) et notez si c'est une victoire ou une défaire pour l'AI ainsi que le nombre de coups moyen, minimal et maximal. Basez-vous sur le nombre de coups optimaux pour résoudre les problèmes pour déterminer un seuil de victoire et de défaite pour l'AI. Précisez votre seuil. **Mettez le tout dans un tableau et analysez vos résultats**. 